In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as optim

In [2]:
#!curl https://www.ars.usda.gov/ARSUserFiles/80400525/Data/SR/SR28/dnload/sr28asc.zip -o ndb.zip && unzip ndb.zip -d ndb

In [3]:
def read_ndb_file(name):
    return pd.read_csv(f'ndb/{name}.txt',
                       encoding='latin1', delimiter='^', quotechar='~',
                       header=None, low_memory=False)

food_des = read_ndb_file('FOOD_DES')

ndb_name2num = {r[2]: r[0] for i, r in food_des.iterrows()}
ndb_num2name = {num: name for name, num in ndb_name2num.items()}

nutr_defs = read_ndb_file('NUTR_DEF')[[0, 3]]
nutr_defs.columns = ['nutr_no', 'nutrient_name']
nutr_defs.loc[nutr_defs.nutr_no == 268, 'nutrient_name'] = 'Energy (kJ)'

nutr_data = read_ndb_file('NUT_DATA')[[0, 1, 2]]
nutr_data.columns = ['ndb_no', 'nutr_no', 'amount_100g']

ingr_stats = nutr_data.pivot(index='ndb_no', columns='nutr_no', values='amount_100g')
assert (ingr_stats.columns == nutr_defs.nutr_no).all()
ingr_stats.columns = nutr_defs.nutrient_name
ingr_stats.index = [ndb_num2name[num] for num in ingr_stats.index]
ingr_stats.index.name = 'food_name'
ingr_stats.fillna(0, inplace=True)

ingr_stats /= 100

In [4]:
ingredients = [
    'Soybeans, mature seeds, raw',
    'Peanut butter, chunk style, without salt',
    'Cocoa, dry powder, unsweetened, processed with alkali',
    'Seeds, chia seeds, dried',
]
LB_G = 453.5924
cost = np.array([
    1.19,
    3.00,
    5.99,
    4.00,
]) / LB_G

pbean_ingr_stats = ingr_stats.loc[ingredients]

energy = pbean_ingr_stats['Energy'].as_matrix()
protein = pbean_ingr_stats['Protein'].as_matrix()
fat = pbean_ingr_stats['Total lipid (fat)'].as_matrix()
w3 = pbean_ingr_stats['18:3 undifferentiated'].as_matrix()
w6 = pbean_ingr_stats['18:2 undifferentiated'].as_matrix()
fiber = pbean_ingr_stats['Fiber, total dietary'].as_matrix()
beans, pb, cocoa, chia = np.eye(4)

In [5]:
def optimize_pbean(optimize='fat'):
    cals_per_serving = 400
    opts = [
        (protein, 'ge', 20),   # require at least 20 g protein
        (w6 - 4*w3, 'le', 0),  # 4:1 omega6 to omega3
        (beans, 'le', 45),     # prevent too much fiber
        (beans, 'ge', 30),     # prevent peanut butter water
        (chia, 'le', 8)        # prevent chia soup
    ]
    
    cost_opt = (cost, 'le', 0.538)  # only somewhat arbitrary
    fat_opt = (fat, 'ge', 22)
    
    if optimize == 'fat':
        c = -fat
        opts.extend([
           cost_opt,
        ])
    elif optimize == 'cost':
        c = cost
        opts.extend([
            fat_opt,
        ])
    if optimize == 'cocoa':
        c = -cocoa
        opts.extend([
            cost_opt,
            fat_opt,
        ])
    minmax, constraints = [], []
    for tgt, ineq, constraint in opts:
        way = 1 - 2*(ineq == 'ge')
        minmax.append(way * tgt)
        constraints.append(way * constraint)
        
    A_ub = np.vstack(minmax)
    b_ub = np.vstack(constraints)
            
    A_eq = energy.reshape(1, -1)
    b_eq = np.array([cals_per_serving]).reshape(1, -1)
    
    soln = optim.linprog(c, A_ub, b_ub, A_eq, b_eq).x
    print_soln(optimize, soln)
    return soln


def print_soln(optimize, soln):
    print(f'Optimizing {optimize}:')
    print(soln)
    print('cost', soln.dot(cost))
    print('fat', soln.dot(fat))
    print('protein', soln.dot(protein))
    print('fiber', soln.dot(fiber))

In [6]:
soln_fat = optimize_pbean('fat')

# soln_cost = optimize_pbean('cost')
# print_soln(soln_cost)

# soln_cocoa = optimize_pbean('cocoa')
# print_soln(soln_cocoa)

Optimizing fat:
[ 41.86312022  23.98250005  15.06979933   8.        ]
cost 0.538
fat 24.7577104088
protein 25.0968757591
fiber 13.0546703849


In [7]:
per_pbean = pbean_ingr_stats.T.dot(soln_fat)
with pd.option_context('display.max_rows', None):
    print(per_pbean[per_pbean != 0] / 3)

nutrient_name
Protein                                 8.365625
Total lipid (fat)                       8.252570
Carbohydrate, by difference             9.984745
Ash                                     1.462401
Energy                                133.333333
Starch                                  0.394112
Sucrose                                 0.334956
Glucose (dextrose)                      0.337354
Water                                   1.573132
Caffeine                                3.918148
Theobromine                           132.312838
Energy (kJ)                           557.919391
Sugars, total                           1.783574
Fiber, total dietary                    4.351557
Calcium, Ca                            64.653482
Iron, Fe                                3.328203
Magnesium, Mg                          84.706994
Phosphorus, P                         183.242894
Potassium, K                          447.203720
Sodium, Na                              3.019183
Zinc, 

## What other nutrients are needed?

In [8]:
PBEANS_PER_DAY = 3

rda = pd.Series({
    'Energy': 2000,
    'Calcium, Ca': 1000,
    'Folate, total': 400,
    'Magnesium, Mg': 400,
    'Niacin': 16,
    'Pantothenic acid': 5,
    'Potassium, K': 4000,
    'Riboflavin': 1.3,
    'Thiamin': 1.2,
    'Vitamin A, RAE': 400,
    'Vitamin B-6': 1.1,
    'Vitamin C, total ascorbic acid': 90,
    'Vitamin E (alpha-tocopherol)': 15,
    'Vitamin K (phylloquinone)': 120,
    'Zinc, Zn': 11,
})

pbean = PBEANS_PER_DAY * per_pbean
need_rda = rda - pbean[rda.index]
need_rda[need_rda > 0]

Calcium, Ca                       418.118664
Energy                            800.000000
Niacin                              0.903530
Pantothenic acid                    3.084871
Vitamin A, RAE                    398.744106
Vitamin B-6                         0.272440
Vitamin C, total ascorbic acid     82.080638
Vitamin E (alpha-tocopherol)        9.234589
Vitamin K (phylloquinone)          59.528237
dtype: float64